In [1]:
from pathlib import Path

from xml.etree import ElementTree as et
from eisenscript_to_xml.lark_inter import Iesxml, parser


In [2]:
test = """set maxdepth 20
10 * { rz 36 }  dbox

rule dbox w 10{
 {x 9.999} box
}

rule dbox w 5{
{ x 10.001  } box
}"""
test = parser.parse(test)
xmlstr = Iesxml().convert(test)
print(xmlstr)

<?xml version="1.0" ?>
<rules max_depth="20">
  <rule name="entry">
    <call rule="dbox" count="10" transforms="rz 36"/>
  </rule>
  <rule name="dbox" weight="10">
    <instance shape="box" count="1" transforms="tx 9.999"/>
  </rule>
  <rule name="dbox" weight="5">
    <instance shape="box" count="1" transforms="tx 10.001"/>
  </rule>
</rules>




Still to do
color stings, adapt parser to allow almost any string - DONE I think
set directives, get parser to read all - Interpreter only understands set maxdepth  - DONE  I think
transforms should be empty on loops with only color transforms
if there's no transforms but still a call count should be 1  DONE


examples of set attributes and camera
set maxobjects  10400
set background #454

set translation [-3.33009 0.0259899 -20]
set rotation [0.147791 -0.237766 -0.960012 -0.822766 -0.568209 0.014066 -0.548833 0.787786 -0.279602]
set pivot [0 0 0]
set scale 0.18957


These seem to go either at the top or between the entry calls and the first rule definition, and to be one per line

TESTS


not implemented but on  
http://structuresynth.sourceforge.net/reference.php
 m  matrix
 fx, fy, fz mirror
 
Use LSystem from Sverchok to test. Strip out all sverchok/blender stuff 
Needs a matrix multiplication currently uses blender mathutils.
there is the standalone version
https://gitlab.com/ideasman42/blender-mathutils

poetry add mathutils
gives lots of errors at fundamental gcc level

The version here
https://github.com/prideout/lsystem/blob/master/Python/LSystem.py
uses euclid!!

make a mash up,
poetry add git+https://github.com/r1chardj0n3s/euclid.git

functions needed are 

mu.Matrix.Identity(4)
mu.Matrix(m) @ mu.Vector(v)
matrix @= mu.Matrix.Translation(mu.Vector((x, 0, 0)))
matrix @= mu.Matrix.Rotation(theta, 4, "X")
matrix @= mu.Matrix.Scale(x, 4, mu.Vector((1.0, 0.0, 0.0)))



In [3]:
from eisenscript_to_xml.generative_art import LSystem

In [4]:
lsys = LSystem(xmlstr, 100)
shapes = lsys.evaluate(0)

In [5]:
len(shapes)

11

Have a working copy of original Structure Synth installed for comparison. Can I get simple stats like number of objects? The obj export doesn't come as objects. just v e f.  
There are other template exports.
the gss file format 
is b followed by 19 numbers = 4 x 4 transform matrix , 3 vector for rgb.
This would be pretty easy to compare with shapes output.
Could also just count rows.
Or get LSystem to have a .gss output.

I'm going to find the output is different - do I care?
Do the structures look the same? Yes.

How would I look at how they differ?
use or rewrite simple importer for gss files into Blender, use sverchok scripted node -DONE

visually the the two gss files do the same thing as the xml generatve art, but the numbers look different
how to determine equivalent mathematically?
Playing in Sverchok  
Matrices are in opposite order, and centered around a different point  

![]("Screenshot from 2022-05-01 16-17-32.png")

Just compare list lengths for testing, as I think GA LSystem is correct.
The GA version is centered on the origin with radius 10 and the boxes centered on the matrix points.
The SS version is offset oddly.
If I export obj from SS and import into Blender one corner of the obj cubes matches the centre of the GA version.
But I can't do a simple overlay of the obj over the gss from SS version???
The gss exporter is odd? The same numbers get sent to the povray file.
I don't get it.

Interstng things happen if you simplify the es as below. The obj's are the same, but the gss are different.



In [6]:
s = ''
for shape, m in shapes[1:]:
    s += ','.join([f'{i:.6f}' for i in m]) + '\n'
print(s)

0.999999,-0.001185,0.000000,0.000000,0.001185,0.999999,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,10.000993,-0.011854,0.000000,1.000000
0.808389,-0.588648,0.000000,0.000000,0.588648,0.808389,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8.084703,-5.887068,0.000000,1.000000
0.308115,-0.951349,0.000000,0.000000,0.951349,0.308115,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.080842,-9.512540,0.000000,1.000000
-0.309806,-0.950800,0.000000,0.000000,0.950800,-0.309806,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-3.097750,-9.507047,0.000000,1.000000
-0.809435,-0.587210,0.000000,0.000000,0.587210,-0.809435,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-8.093539,-5.871510,0.000000,1.000000
-1.000000,0.000593,0.000000,0.000000,-0.000593,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-9.998998,0.005926,0.000000,1.000000
-0.808738,0.588169,0.000000,0.000000,-0.588169,-0.808738,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-8.0

In [7]:
fn = 'ring_xml3.gss'
with open(fn, 'w') as f:
    f.write(s)

``` python
# gss_matrices.py scripted node lite
"""
in dummy  s
out matrices      m
out test1          s
"""

from mathutils import Matrix

fn = '/home/elfnor/beasthoard/gits/eisenscript_to_xml/notebooks/ring_es.gss'

with open(fn, 'r') as f:
    data = f.readlines()

mats= []
for d in data:
    m = d.strip().split(' ')
    m = [float(n) for n in m[1:]]
    m = [m[n:n+4] for n in range(0, len(m), 4) ]
    mats.append(Matrix(m).transposed())

matrices = mats
test1 = [len(data)]
```

In [8]:
test = """set maxdepth 10
10 * { rz 36 }  dbox
rule dbox {
 {x 10 }  box
}
"""
test = parser.parse(test)
xmlstr = Iesxml().convert(test)
print(xmlstr)

<?xml version="1.0" ?>
<rules max_depth="10">
  <rule name="entry">
    <call rule="dbox" count="10" transforms="rz 36"/>
  </rule>
  <rule name="dbox">
    <instance shape="box" count="1" transforms="tx 10"/>
  </rule>
</rules>



\#define commands 

#define angle 36

`<constants  angle="36" />`

and 
`<call rule="dbox" count="10" transforms="rz {angle}"/>`

need to redefine VALUE in grammar to include names

does svechok version handle constants? Can't see it in code?
There was a preprocessr that substitued?
Happens in  `xml_text_format(self)` which I've not included here!




In [9]:
test = """#define angle 36
set maxdepth 20
10 * { rz angle}  dbox
rule dbox {
 {x +10.0 }  box
}
"""
test = parser.parse(test)
xmlstr = Iesxml().convert(test)
print(xmlstr)

UnexpectedCharacters: No terminal matches '#' in the current parser context, at line 1 col 1

#define angle 36
^
Expected one of: 
	* NUMBER
	* NAME
	* SET
	* LBRACE


In [ ]:
lsys = LSystem(xmlstr, 100)
shapes = lsys.evaluate(0)
len(shapes)

In [ ]:
xml_tree = et.fromstring(xmlstr)
type(xml_tree)

In [ ]:
import xml
isinstance(xml_tree, et.Element)

In [ ]:
fn = Path('/media/elfnor/BeastHoard/gits/eisenscript_to_xml/tests/test_es/Examples/Torus2.es')
with open(fn, 'r') as f:
    es_text = f.read()
test = parser.parse(es_text)
xmlstr = Iesxml().convert(test)
lsys = LSystem(xmlstr, 5000)
shapes = lsys.evaluate(0)



In [ ]:
print(xmlstr)

In [ ]:
es_files = Path('/media/elfnor/BeastHoard/gits/eisenscript_to_xml/tests/test_es').rglob('*.es')
for fn in es_files:
    print(str(fn), end='')
    with open(fn, 'r') as f:
        xml_text = f.read()
    try:
        test = parser.parse(xml_text)
    except: 
        print(' parser error ', end='')
    try:
        xmlstr = Iesxml().convert(test)
    except:
        print(' interprter error ', end='')
    try:
        lsys = LSystem(xmlstr, 100)
        shapes = lsys.evaluate(0)
    except:
        print(' xml lsystem error ', end='')
    print()
    print()

        

In [ ]:
es_text = """#define angle 36
set maxdepth 20
10 * { rz angle}  dbox
#define _radius 10
rule dbox {
 {x _radius }  box
}
"""

# rough go at #define preprocessor without grammar
# assume #define happens before use
# is naive about replacement
es_lines  = es_text.splitlines()
# print(es_text)
format_dict = {}
new_lines = []
for line in es_lines:
    if line.startswith('#define'):
        print(line)
        parts = line.split()
        if len(parts)  == 3:
            format_dict[parts[1]] = parts[2]
    else:
        print('old ', line)
        for key, item in format_dict.items():
            line = line.replace(key, item)
        new_lines.append(line)
        print('new ',line)
        print()
print('\n'.join(new_lines))



In [ ]:
test = """set maxdepth 20
10 * { rz 36 }  dbox

rule dbox w 0.1{
 {x 9.999} box
}

rule dbox {
 {x 9.999} box
}
rule dbox w 0.5 {
{ x 10.001  } box
}"""
test = parser.parse(test)
xml_text = Iesxml().convert(test)
print(xml_text)

<?xml version="1.0" ?>
<rules max_depth="20">
  <rule name="entry">
    <call rule="dbox" count="10" transforms="rz 36"/>
  </rule>
  <rule name="dbox" weight="0.1">
    <instance shape="box" count="1" transforms="tx 9.999"/>
  </rule>
  <rule name="dbox">
    <instance shape="box" count="1" transforms="tx 9.999"/>
  </rule>
  <rule name="dbox" weight="0.5">
    <instance shape="box" count="1" transforms="tx 10.001"/>
  </rule>
</rules>



In [ ]:
# rough go at a postprocessor to convert float weights to integer
xml_tree = et.fromstring(xml_text)
rules = xml_tree.findall("rule")
weights = []
for rule in rules[1:]:
    w = float(rule.get('weight', 1))
    weights.append(w)
weights

[0.1, 1.0, 0.5]

In [ ]:
wscale = int(1 / min(weights))
new_weights = [int(wscale * w) for w in weights]

In [ ]:
for w, rule in zip(new_weights, rules[1:]):
    rule.set('weight', f'{w}')

In [ ]:
import xml.etree.ElementTree as et
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = et.tostring(elem, "utf-8")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [ ]:
print(prettify(xml_tree))

<?xml version="1.0" ?>
<rules max_depth="20">
  
  
  <rule name="entry">
    
    
    <call rule="dbox" count="10" transforms="rz 36"/>
    
  
  </rule>
  
  
  <rule name="dbox" weight="1">
    
    
    <instance shape="box" count="1" transforms="tx 9.999"/>
    
  
  </rule>
  
  
  <rule name="dbox" weight="10">
    
    
    <instance shape="box" count="1" transforms="tx 9.999"/>
    
  
  </rule>
  
  
  <rule name="dbox" weight="5">
    
    
    <instance shape="box" count="1" transforms="tx 10.001"/>
    
  
  </rule>
  

</rules>

